In [ ]:
# https://velog.io/@seolini43/%EC%9D%BC%EC%83%81%EC%97%B0%EC%95%A0-%EC%A3%BC%EC%A0%9C%EC%9D%98-%ED%95%9C%EA%B5%AD%EC%96%B4-%EB%8C%80%ED%99%94-BERT%EB%A1%9C-%EC%9D%B4%EC%A7%84-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0%ED%8C%8C%EC%9D%B4%EC%8D%ACColab-%EC%BD%94%EB%93%9C

In [1]:
import tensorflow as tf
import torch
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import random
import time
import datetime
import gc
gc.collect()
torch.cuda.empty_cache()

n_devices = torch.cuda.device_count()
print(n_devices)
for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
GeForce RTX 3090


In [2]:
data_ad = ["이충동 즐거운점핑", "재밌고 신나게 점핑해요", "정말 이쁘게 살빠져요", "남사면 마이그린", "식물농장", "식물도매농장", "식물화분 1,400", "평택시 미리내스칼프", "탈모잡는 진짜 전문가", "할인이벤트+제품무료증정", "서정동 나우어학원", "돌아온 겨울방학 대학생", "영어 특강 예약 시작", "평택시 교토화로", "프리미엄 소고기 화로구이", "살살녹는 교차숙성비법", "대표 소갈비살 23,000", "#핫플예정", "광고", "아까 회의에서 뭐라고.. 했더라?","이것은 위대한 시작! 듄","스물 아홉이 된 너와 나의 기록","ART-9세종 뮤지컬","국립오페라단 라 보엠","2021 한국의 소비자보호 우수기업","한정 특가 EVENT","뮤지컬 풍월주","국립오페라단 라 트라비아타","MINO 2021.12.15 PM 8:00","후원 LIVE","2020 뮤지컬 MOZART!","스페셜 에디션","12월25일~27일7:00PM","누구나 자유롭고", "안전하게 데이터 활용","통신 빅데이터 플랫폼","휴먼, 인공지능이", "궁금하십니까?","SEF2021에서 알려드립니다!",
"네이버 커넥트재단", "문과생 집중!", "how to be 개발자?", "SEF 2021", "연말 한정 최대 ~ 51% 특가세일! 지금이기회야", "닥터피엘", "[1,300만개 판매] 국민 샤워기", "12만개가 넘는 극찬후기 ! 너무 좋잖아 ~", "말도 안되지만 오후 3시까지 삼성 노트북 전원 증정", "7년 연속 영어회화 판매 1위 스피킹 맥스", "MYCAR 오너평가 서비스", "내 차 평가하고", "1,000원 받으세요!", "내 차 평가하기", "평택시 베이컨시", "분위기 좋은 카페", "브런치와 시그니쳐 맛집", "아메리카노 4,000", "평택시 꿈을만드는공방 본사", "핸드메이드 선물공방", "드림캐쳐,썬캐쳐,마크라메", "리뷰 59", "신장동 베스트프렌즈 동물병원", "외과특화진료", "영어진료,건강검진,중성화", "초진비 25,000", "네이버 해피빈","지역사회 이웃에게 사랑을 전합니다","WINTER FESTIVAL","라그나크와 함께하는 겨울나기","NAVER Cloud SUMMIT","초거대 AI 하이퍼 클로바를 만나보는 시간","무료 사전등록","구매 고객 전원 100% 선물 증정","후원 LIVE","강아지 기관지 영양제 닥터바이 브레스","나이가 들수록 심해지는 기관지 협착","기관지에서 호흡기까지 하루 한포로 관리하자!",
"브랜드 위크", "아우디 코리아", "브랜드관 바로가기", "AD", "시승기","평택시 스카이휘트니스", "서정동 서정리역 헬스장", "개인PT,그룹PT근육증가","셀프 랩핑용! 자동차등 이쁘게 꾸밀수있는 시트지!","ICT 전문가의 도움이 필요한 당신!","CT이노베이션스퀘어 ICT 상시 멘토링 프로그램에 참여하세요!","다회용 토이카메라 세트 증정","너를 위한 선물에 나를 위한 큰 적립 혜택!","풍성한 에너지 절대지켜","나만의 PICK아우터에 투표해주세","매일 새로운 제주특가 LIVE","스트레스 없는 일상의 시작","산지에서 바로보내드리는 샘물과메기","TOP10과 함께 하는 LIVE SHOW","1개만 사도 무료배송","취창업 정부지원 선착순 모집!","7분 신청 가능, 주택담보대출","프리바이오틱스 3+3 세트 반값 할인","현대해상다이렉트 이륜차보험","바몽더브랜드 맨투맨 SALE 14%","이카운트 ERP 월 4만원","미션 참여하면 아이패드 겟!","컬러마스크 이 가격?","온라인판매 사관학교(1:1멘토시스템)"
"2021년 블로그", "역대급 빅데이터", "대.공.개", "놀고 쉬는 경북투어","경북으로 간다", "클릭하고 기부콩 1개 받기", "네이버 해피빈", "가볼까", "기획전", "바로가기>", "미세먼지", "'좋음'", "N리빙", "공간100P!CK", "우리 집 소개하면", "감성 공간마다", "N Pay 3만원", "Lifelog.Blog", "17년 차 카피라이터", "루나파크의", "창작 비법은 OOO!", "자세히 보기", "셀러 지원 프로그램", "#내 마켓 알리기 서포트", "참가자 모집", "(~12/13)", "행복상자", "선물 작전!", "post.naver.com", "브랜O, 지그재O 이미 완판", "뜨거운 반응 난리난 신상 모자 실물",  "오픈저널","더패치", "디지털미디어팀에게", "제보해주세요.", "제보메일 world_biz@naver.com", "The Patch", "4주간 답변 미션 달성하면", "최대 1만 원의 선물을 드려요!", "문제, 롯데ON", "할인", "첫구매50%할인", "문제 쇼핑의 모든 것, 이번주만 20%쿠폰팩+7%카드할인+1만점 적립찬스!", "온라인 문제사이트, 티메이트", "이벤트", "모의고사 할인이벤트 진행중", "원하는 모의고사 응시 후 종합분석표와 오답노트까지 바로 받아보자!",
"누구나 자유롭고", "안전하게 데이터 활용", "kt", "통신", "빅데이터 플랫폼", "NAVER Cloud SUMMIT", "초거대 AI", "하이퍼 클로바를", "만나보는 시간", "무료 사전등록>", "구독하기", "공감하기", "좋아요", "싫어요", "카테고리의 다른 글", "Related Articles", "전체", "이미지", "뉴스", "동영상", "쇼핑", "더보기", "도구", "앱", "관련 검색어", "1:1", "지식iN에 물어보기", "지식iN 더보기", "전체", "블로그", "카페", "VIEW", "VIEW 더보기", "최고 인기", "인기", "최신순", "인기순", "등록순", "한정된 시간 동안 ₩17,000 이상의 강의를 만나보세요", "지금 구매", "장바구니", "기획전!", "TOP", "공유", "추천", "찜", "바로가기", "이동하기", "클릭하기", "up to", "~55%", "관심주제 설정", "자세히보기>", ">", "<", "응원해요", "축하해요", "기대해요", "놀랐어요", "슬퍼요", "찬성", "반대", "리트윗", "마음에 들어요", "전체영상", "날짜별 영상", "테마별 영상"]

In [3]:
data_nonad = ["윤석열 국민의힘 대선 후보가 10일 자신의 외가가 있는 강릉 중앙시장을 찾아 “강릉의 외손이 무도하고 무능한 정권을 반드시 교체해내겠다”고 말했다.",
"윤 후보는 이날 오후 6시17분께 강릉 중앙시장을 찾아 자신을 기다리던 시민 수백명을 향해 “강릉의 외손이 강릉에 왔다”며 ‘만세’하고 손을 흔들었다.",
"이어 “이 거리가 저의 외가가 있던 곳이고 여기 중앙시장이 저 어릴 때 저의 할머니가 가게하시던 곳”이라며 “강릉 시민 여러분 감사하다”고 말했다.",
"윤 후보는 선거대책위원회 출범식을 한 뒤 첫 지방 일정으로 강원을 택했다.",
"더불어민주당이 내년도 가계대출 증가율을 4~5% 정도로 관리해 달라고 금융당국에 요청했다.",
"전세대출 등 서민, 실수요자들이 최대한 영향을 받지 않도록 금융정책을 운영해 나가기로 했다.",
"강릉시장 입구에서 ‘어·대·윤’(어차피 대통령은 윤석열) 플래카드를 들고 있던 장옥화(61)씨는 “강릉은 윤석열 고향이니 다 윤석열”이라며 “홍제동 힐스테이트 부녀회 회원이 12명인데 전부 다 나왔다”고 말했다.",
"빨간 글자로 ‘윤석열’ 세 자를 쓴 플래카드를 들고 있던 경포 지역 국민의힘 당원 이강수(65)씨는 “80% 이상이 윤 후보를 지지한다”고 말했다.","한국과학기술원(KAIST)과 사단법인 우리들의미래는 이날 온라인으로 제8회 서울 기후·에너지 컨퍼런스를 개최했다. 대통령 후보들은 사전 녹화 영상 메시지를 통해 각자의 기후·환경·에너지 정책 공약을 밝혔다.",
"신고를 받고 출동한 경찰은 인근 건물에 숨어 있던 A씨를 발견하고 오후 2시 50분쯤 현행범 체포했다.",
"최근 코로나19 상황이 지속되고 비대면 거래가 확대되면서 온라인이나 전화 등을 통한 금융사기 피해가 늘고 있습니다. 특히 대출이 급한 저신용자, 구직중인 사회초년생, 전업주부 등이 주요 타깃입니다. 그 중 가장 주의해야 할 점이 있는데요, 바로 '명의대여'입니다.",
"그런데 일부 지자체 방역 관련 조례는 이보다 더 구체적으로 영업시간이 아닌 학원 수업시간 제한만을 규정하고 있어서 학원과 교습소가 보상을 제대로 받지 못해왔습니다.",
"코로나19 사태가 터지면서 온라인 수업 등으로 전환하며 감염 예방에 대응해 왔지만, 코로나 이전 경영 회복은 아직 못하고 있습니다.",
"특히 명품 '오픈런'을 비꼬는 김희애 광고는 인기를 끌었다. 김희애는 광고에서 굳이 줄을 서지 않고도 온라인으로도 충분히 명품을 살 수 있다는 점을 강조했다.윤 후보가 이동하는 곳마다 ‘윤석열’을 연호하는 사람들이 십수명씩 나타났다.",
"몽골 국적의 여중생을 집단 폭행한 또래 가해자들을 엄벌해달라는 내용의 청와대 국민청원이 정부 답변 기준인 20만명 이상의 동의를 받았다.",
"오미크론 변이가 국내에 확산하기 시작한 것도 추가 접종 간격을 단축한 또 다른 배경이다. 화이자는 기존 백신을 추가 접종하면 오미크론 변이에 대해 무력화 효과가 있다는 예비 분석 결과를 발표하기도 했다.",
"지난해 임대차법 시행 이후 갱신권을 쓴 재계약이 늘면서 신규 전세 수요가 크게 줄었기 때문입니다.",
"이후 이 대표와 함께 강릉커피거리를 방문하고 흉기난동범을 맨손으로 제압한 전중현·변정우씨를 면담한다.",
"10일 금융감독원 전자공시시스템에 따르면, 류영준 대표는 지난 8일 시간외매매를 통해 보유 중인 카카오페이 주식 23만주를 매각했다. 1주당 매각 대금은 20만4천17원으로, 총 매각 대금은 470억원에 달한다.",
"질병청에 따르면 이날 0시 기준 인구 대비 추가 접종 완료율은 10.3%로 약 529만명이 추가 접종을 마쳤다. 이 중 60세 이상 고령층은 약 350만명이다.",
"포스코가 지주회사인 ‘포스코홀딩스’와 철강사업회사인 ‘포스코’로 나뉜다.",
"반면 평가원 측은 문항의 조건이 완전하지 않더라도 학업 성취 수준을 변별하기 위한 평가 문항으로서 타당성이 유지된다고 맞섰다. 재판부는 사안의 중대성을 고려해 오는 17일 1심을 선고하겠다고 밝혔다.",
"기존 포스코는 지주회사로 바뀌면서 상장사로 유지되고, 지주사가 지분 100%를 소유한 비상장 철강사업회사 ‘포스코’가 물적분할된다.",
"숨진 여성의 가족으로부터 실종신고를 받은 경찰은 A씨를 임의동행해 조사하는 과정에서 범행을 자백받은 것으로 알려졌습니다.",
"대입 일정에 변수가 생기면서 교육부와 한국교육과정평가원은 이날 후속 대입 일정과 관련해 한국대학교육협의회, 대학들과 협의했다. 교육부 관계자는 “대입 현장의 혼란을 최소화하자는 데 관계자들의 의견이 모였다”고 말했다.",
"포스코가 지주회사 체제로 바꾸는 이유는 신사업 발굴·육성에 힘을 싣고 시장 가치를 제대로 평가받기 위해서다.",
"여기서 끝이 아닙니다. 사기범은 할부대출금을 약 2개월 정도 대신 납부해주면서, 명의 대여자를 소개해주면 사례금을 지급하겠다고 유인하죠. 피해자는 친척이나 지인 등을 추가로 사기범에게 소개하기도 합니다.",
"윤석열 국민의힘 대선후보 측은 10일 윤 후보가 검찰총장으로 재직하던 시절, 법무부로부터 받은 직무정지 처분을 취소해달라며 낸 소송이 ‘각하’결정을 받은 것에 대해 “역사와 국민의 판단은 이미 내려졌다. 정권교체로 종지부를 찍겠다”고 말했다.",
"아울러 “최근 철강 사업에서 역대 최대 실적을 올릴 뿐 아니라 국내외 이차전지소재, 수소 사업 등에서 적극적으로 주도권을 확보하고 있음에도 코스피 상승과는 달리 포스코의 주가는 저평가된 상황”이라며 “철강 중심 기업이라는 인식 때문에 신성장 사업에 대한 가치를 적절히 평가받지 못하고 있다는 의견도 그룹 안팎에서 대두됐다”고 밝혔다.",
"미국의 대통령과 국방장관이 바뀐 것이 한미 간의 군사 현안 논의 과정과 결과에서 명백하게 큰 차이를 만들어냈습니다.",
"딸에게 쫓겨난 노모는 이곳 집 앞에서 약 1시간 반 정도 방치된 것으로 전해집니다.",
"손 전 의원은 전날인 9일 페이스북에 김씨의 과거와 현재 사진을 나란히 올린 뒤 “얼굴이 변했다는 것은 이미 알고 있었지만 자세히 보니 눈동자가 엄청 커져 있다”며 김씨의 성형 의혹을 제기하는 듯한 글을 남겼다.",
"정부가 검토 중인 '특단의 조치'에는 사실상 과거 사회적 거리두기와 같은 강력한 규제 방안도 고려되고 있는 것으로 보인다.",
"세월호 참사가 있은 뒤로 7년 8개월 동안 끊어졌던 인천항과 제주항 사이 뱃길이 오늘(10일) 다시 열렸습니다. 조금 전에 신뢰 그 이상이라는 뜻을 담은 비욘드 트러스트호가 인천항을 떠났는데요. 배에 타 있는 취재기자를 연결해 보겠습니다.",
"양구에서는 공무원 1명이 확진돼 일부 부서를 폐쇄하고 코로나 진단 검사가 진행되고 있습니다.",
"다만 구속영장 청구 하루 만에 극단적 선택을 해, 검찰 수사가 영향을 준 거 아니냔 추측만 나옵니다.",
"그렇습니다. 당장 '이재명의 민주당'이 뭐가 달라진 것이냔 얘기가 나오겠죠. 특히 이 후보는 당시에도 신뢰를 강조하며 무공천을 주장했습니다.",
"'약자와의 동행'을 모토로 내건 국민의힘 윤석열 대선 후보가 오늘은 사회복지 비전 선포 대회에 참석했습니다.",
"직책도 아니고 특보니까. 다양한 분들이 특보로서 그런 일을 할 수 있지 않겠습니까? 국민들에게 보여주는 상징적인 자리는 아니고 여러 분들이 특보로서 선거운동을 하는 거니까."
"롯데 자이언츠에서 2년 간 배터리 코치를 맡았던 최현(33·행크 콩거) 코치가 메이저리그로 돌아간다.",
"2010~2016년 빅리그에서 활약했던 최현은 은퇴 후 지도자의 길로 들어섰다.",
"지난 2020시즌을 앞두고 롯데의 배터리 코치로 부임했다.”,”2019년 역대 최초 100폭투를 기록하는 등 부진했던 롯데 안방을 단숨에 개선시켰다는 평가를 받았다.",
"김준태(현 KT), 지시완, 정보근, 안중열, 손성빈 등 여러 포수들의 성장을 이끌었다.",
"올해는 래리 서튼(51) 감독이 가족의 코로나19 감염 여파로 자가격리를 하면서 최 코치가 임시 사령탑을 맡기도 했다.",
"성적은 3승 3패. 후반기에는 배터리 코치와 함께 수석 코치 역할을 겸임했다.",
"올 시즌을 앞두고 재계약을 맺어 내년까지 롯데와 함께 하기로 했으나 메이저리그의 오퍼를 받으면서 거취에 변동이 생겼다.",
"최근 KIA 타이거즈행 소문이 돌고 있는 '외야 FA 최대어' 나성범(33)이 오랜만에 골든글러브 시상식에 참석해 근황을 전했다.",
"올해 144경기 타율 0.281, 33홈런 101타점, OPS 0.844로 준수한 성적을 남긴 나성범은 외야수 골든글러브 후보에 올랐다.",
"만약 이번에 수상을 하게 된다면 2014년, 2015년 이후 3번째 골든글러브다.",
"그는 2012년 2라운드 전체 10번으로 입단해 지난 9년간 NC의 대표 프랜차이즈 스타로 활약했다.",
"지난 2020시즌을 앞두고 삼성 구자욱은 등번호를 바꿨다.",
"65번에서 5번으로 교체하고 새 시즌을 맞았다.",
"뜨거운 인기남 구자욱의 유니폼을 구입했던 팬들 사이에 큰 화제를 모았다.",
"설만 무성했을 뿐 구체적 이유는 밝혀지지 않았다.",
"그리고 2년 후, 골든글러브 수상은 현실이 됐다.",
"10일 삼성동에서 열린 시상식에서 구자욱은 외야 부문에서 생애 첫 골든글러브 수상자로 호명됐다.",
"행사 후 인터뷰에서 구자욱은 2년 전 묻어둔 비밀을 비로소 털어놓았다.",
"바로 김한수 전임 감독의 등번호를 물려받은 사연이 숨어 있었다.",
"김한수 전 감독을 떠올리며 눈시울을 붉힌 구자욱.",
"그는 결국 '5번의 약속'을 멋지게 지켰다.",
"이제는 지킬 일만 남았다.",
"더 나아가 대한민국 최고 선수로의 도약을 꿈꿀 만한 특급 선수.",
"골든글러브 수상은 5번 구자욱이 맞이할 영광의 첫 걸음이다.",
"주인공이 상대를 향해 뱉는 날 선 대사가 어쩐지 익숙하고, 난장판으로 치닫는 주인공들의 상황을 보며 나도 모르게 지난 기억을 뒤적일 때.",
"그런 영화들은 픽션에서 현실이 된다.",
"전종서, 손석구 주연의 <연애 빠진 로맨스>가 바로 그런 유형의 영화다.",
"술자리에서 주고받는 대사엔 내숭이 하나도 없고, 친구들과 이성 관계를 논하며 팝콘처럼 거침없는 대사를 팡팡 쏟아내는 인물들을 보며 데자뷔 효과를 느낀 관객이 한둘은 아닐 거다.",
"통통 튀는 발칙함으로 관객에게 색다른 자극을 전한 <연애 빠진 로맨스>처럼 저만의 ‘매운맛’ 포인트로 관객의 호응을 얻었던 연애 영화를 한자리에 모았다.",
"엄청난 강박 증세의 소유자 이건은 우연히 만난 레나와 정신 못 차릴 정도로 얼얼한 사랑에 빠진다.",
"문제가 있다면 레나를 만나기 전 폰 섹스를 하며 악덕 업체 일당과 엮였고, 그들이 레나와 이건의 사랑을 방해한다는 것.",
"그들에게서 벗어나랴, 사랑을 지키랴 정신없는 이건은 모든 상황을 극복할 수 있을까?",
"<펀치 드렁크 러브>의 주인공들은 눈에 띄는 결함을 지니고 있다.",
"완벽하지 않은 이들이 너무나 완벽한 사랑을 펼쳐내는 과정은 보는 이마저 사랑에 빠지게 만든다.",
"제목처럼 정신을 못 차릴 정도로 사랑에 흠뻑 빠진 이들의 감정은 폴 토마스 앤더슨의 독특한 연출을 통해 시각화되어 나열된다.",
"강다니엘 측의 안일한 대응이 녹화장을 마비시켰다.",
"강다니엘 측은 주최 측의 콜타임을 핑계로 삼았지만, 대부분의 소속사 관계자들이 PCR 결과를 듣고 참석했다.",
"CJ 측의 대응도 도마 위에 올랐다.",
"워너원은 이날 오후 서울의 한 병원에서 급속 테스트를 진행했다.",
"오후 8시경에 결과가 나올 예정이다.","현재 전원 자택에서 격리하고 있다.",
"무엇보다 가장 큰 피해는 팬들에게 돌아갔다.",
"이날 현장에는 약 500명의 팬이 새벽부터 모였다.","이주용 회장에게는 최초라는 수식어가 많이 붙는다.",
"그렇게 귀국길에 오른 그는 한국생산성본부 산하 한국전자계산소의 소장을 맡아 국내에 처음으로 컴퓨터를 도입했다.",
"대만 위생복리부는 이 연구원의 코로나19 감염을 확인한 후 그와 접촉한 직장동료 및 친구 94명의 신원을 확인하고 격리 조치했다.",
"조선 업계도 강성 노조 위원장 당선으로 분위기가 어수선하다.",
"이번 조치로 연내 부스터샷 대상자는 1699만 명에서 2641만 명으로 늘어난다.",
"이번 조치는 2차 접종을 마친 고령층에서 돌파감염과 사망이 급증한 데 따른 것이다.",
"유럽연합(EU)의 의약품 규제 당국인 유럽의약품청(EMA)은 접",
"종을 완료한 뒤 3개월만 지났어도 부스터샷을 맞는 것이 효과적이라고 9일(현지 시간) 밝혔다.",
"정부는 18세 미만 소아·청소년 대상 부스터샷은 아직 검토하지 않",
"지만 해외 동향을 주시하고 있다고 밝혔다.",
"정부는 백신을 맞고 사망한 경우 인과성 근거가 충분하지 않아도 내년부터 위로금 5000만 원을 유가족에게 지급하기로 했다.",
"대만에서 한달여 만에 처음으로 코로나19 지역감염자가 발생했다.",
"이 여성은 최근 해외여행 이력이 없으며, 모더나 백신 2회 접종을 완료한 상태다.",
"대만 당국은 이 여성이 코로나19 새 변이 오미크론이 아닌 기존의 영국발 델타 변이에 감염된 것으로 보고 있다.",
"최근 대선 관련 '썰'을 집중해 풀어내고 있는 진중권 전 동양대 교수가 10일에는 추미애 전 법무부 장관과 유시민 전 노무현재단 이사장을 '장외 윤석열 선대위'라고 표현했다.",
"검찰 수사를 받던 유한기 씨가 숨진 채 발견됐는데 거기에 대해서는 어떻게 생각하십니까?",
"김종인 위원장께서 손실보상 100조 얘기하셨는데요, 후보님은 예전에 50조 얘기하셨었잖아요.",
"정부조직 개편은 여야가 협의를 해서 결정을 해야 되는 문제이지만 우리가 종전의 정부조직체계에서 새로이 미래를 위해서 우리가 다뤄야 할 아젠다 중심으로 정부조직이 개편돼야 되는 것은 맞는 방향입니다.",
"저희는 특검을 하자는 얘기를 벌써 부산저축은행 가지고 하더라도 하자는 얘기를 벌써 진작에 꺼내놨고 지금 민주당에서는 법안 자체를 올리지를 않고 있어요.",
"아직 선거가 본격화하지 않은 기아(000270)에도 전운이 감돌고 있다.",
"오는 16~17일 1차 투표에서 총 3명의 후보가 맞붙는 가운데 2명이 강성으로 분류된다. 기아에도 강성 지도부가 들어서면 현대차그룹 내 강성 노선이 주류가 된다.",
"[서울=뉴시스] 오동현 기자 = 카카오게임즈는 남궁훈 각자대표가 카카오의 미래 이니셔티브 센터장으로서 역량을 집중하기 위해 대표이사 직을 사임한다고 10일 밝혔다.",
"지난 1일 카카오 미래 이니셔티브 센터장에 선임된 남궁훈 전 대표는 김범수 의장과 함께 카카오 공동체의 미래 10년(beyond mobile)을 위한 글로벌 시장 공략과 미래 먹거리 발굴 등 통합 로드맵 구상과 실행에 더욱 속도를 낼 예정이다.",
"카카오게임즈는 조계현 단독 대표이사 체제로 전환하고 새로운 출범을 알렸다.",
"이와 함께 한상우 CSO가 수석 부사장으로, 안양수 PC기술본부장이 CTO로 승진했다.",
"카카오게임즈는 2016년 남궁훈, 조계현 각자대표 체제로 게임 사업을 시작해 플랫폼과 게임 유통, 개발까지 수직 계열화를 구축하며 지난해 기업공개와 올해 '오딘: 발할라 라이징'의 흥행을 성공시켰다．",
"네이버 등 포털업체들이 불법촬영물 유통방지를 위한 필터링 조치를 본격 시작했다.",
"디지털 성범죄물 유통을 막기 위한 이른바 ‘n번방 방지법'(개정 전기통신사업법 및 정보통신망법)이 10일부터 시행되면서다.",
"카카오는 이날부터 ‘오픈채팅 그룹채팅방’에 대해 불법촬영물등의 ",
"유통방지와 이용자보호를 위한 기술적·관리적 조치가 적용된다고 밝혔다.",
"오픈채팅 그룹채팅방의 동영상, 움직이는 이미지, 압축파일에 대해서는 필터링이 이뤄지게 되며 불법촬영물등의 유통에 대해서는 사전 경고 조치, 기술적 조치에 대한 로그기록의 보관이 이뤄진다.",
"네이버도 이날부터 불법촬영물 등에 대한 유통 방지를 위한 기술적·관리적 조치가 적용된다고 밝혔다.",
"네이버는 이미 AI 기반 영상·이미지 필터링 기술(X-eye)을 통한 음란·불법 게시물의 유통 차단, ‘디지털 성범죄 예방 캠페인’ 및 '그린인터넷 캠페인' 운영 등 안전한 인터넷 환경을 만들기 위해 다양한 캠페인을 진행하고 있다고 덧붙였다.",
"삼성전자가 스마트폰을 담당하는 사업부 이름을 무선사업부에서 MX(Mobile Experience·모바일 경험)사업부로 바꾼다. 1995년 이후 26년 만이다. 대표이사 세 명을 모두 교체하는 쇄신 인사에 이은 첫 후속 조치다.",
"삼성전자는 10일 미래 지향 가치를 반영하는 동시에 가치 있는 고",
"객 경험을 최우선으로 하겠다는 뜻에서 무선사업부 이름을 바꿨다고 밝혔다.",
"새로운 이름 ‘MX’에 스마트폰부터 태블릿, PC, 웨어러블 기기 등 다양",
"한 제품을 통해 소비자가 최적화된 경험을 즐길 수 있도록 하겠다는 의지를 담았다고 설명했다.",
"삼성경제연구소도 이날 삼성글로벌리서치로 사명을 변경한다고 발표했다.",
"글로벌 경영환경을 체계적으로 분석해 삼성의 글로벌 초일류화에 기여하는 싱크탱크가 되겠다는 뜻에서 이름을 바꾸게 됐다고 설명했다.",
"삼성전자의 세트 부문 수장에 올라선 한종희 부회장이 무선사업부 명칭을 'MX사업부'로 변경하며 새로운 '갤럭시 생태계'를 구축하기 위한 움직임에 본격 착수했다.",
"모바일, PC, 웨어러블 제품을 넘어 가전, TV까지 아우르는 삼성만의 생태계를 조성함으로써 미래 선제 대응과 함께 애플을 넘어서겠다는 의지로 읽힌다.",
"카카오모빌리티가 GS리테일의 주차장 운영업체 'GS파크24'를 인수",
"하면서 주차 시장의 경쟁이 뜨거워지는 모양새다.",
"이번 인수로 직영 주차장 수를 크게 늘린 카카오모빌리티는 현재 서비스 중인 주차 서비스 '카카오 T 주차'에 힘을 더욱 쏟을 것으로 전망된다.",
"카카오모빌리티는 이번 인수로 주차 사업을 하는 다른 업체들과 직영 주차장 수에서 어느 정도 숫자를 맞추게 됐다.",
"EV(전기차) 충전, 세차, 경정비 등 주차장을 거점으로 하는 자동",
"차 운전자 대상 서비스를 비롯해 물류, 공간 콘텐츠 등 다양한 혁신 서비스와 기술을 테스트해 나갈 계획이다.",
"지난 2017년부터 주차 사업을 시작한 카카오모빌리티는 코엑스, 에버랜드, 센터필드 등의 주차장 운영을 맡으면서 사업을 확대해 나갔고 이번 인수로 양적인 면에서도 몸집을 불리는 데 성공했다.",
"펄어비스(대표 정경인)의 신작 ‘도깨비(DokeV)’의 공식 주제가 ‘락스타(ROCKSTAR)’의 뮤직비디오가 오는 11일 열리는 ‘2021 엠넷 아시안 뮤직 어워즈(MAMA)’에 나선다. 게임이 아닌 ‘케이팝(K-POP)’으로 글로벌 팬들을 사로잡을 전망이다.",
"MAMA는 CJ ENM이 주관하고 매년 세계 각국의 아티스트이 참여하는 최고의 케이팝 음악 시상식이다. MAMA에 게임 콘텐츠를 선보이는 건 이번이 처음이다.",
"‘도깨비’ 주제가 ‘락스타’는 유명 걸그룹 ‘있지(ITZY)’의 ‘달라달라’를 작곡한 ‘별들의전쟁 *(GALACTIKA *)’이 참여한 곡이다. 케이팝 특유의 중독성 있는 음악으로 호평을 얻고었다. 지난 8월 유럽 게임전시회 ‘게임스컴’에서 플레이 영상이 공개됐을 당시 음악에 대한 호평도 많았다.",
"[서울=뉴시스]김예지 인턴 기자 = 그룹 에스파(aespa)가 미국 토크쇼 '더 닉 캐넌 쇼(The Nick Cannon Show)'에 K팝 아티스트 최초로 출연했다.",
"10일 SM엔터테인먼트에 따르면 에스파는 이날 오전 방송된 미국 폭스채널(Fox 채널) '더 닉 캐넌 쇼'에 등장해 토크와 함께 첫 미니앨범 타이틀 곡 '새비지(Savage)' 퍼포먼스를 선보였다.",
"한편, 에스파 첫 미니앨범 '새비지'는 미국 빌보드의 메인 차트인 '빌보드 200'에 20위로 첫 진입해 K팝 걸그룹 첫 앨범 사상 최고 순위를 기록한 바 있다.",
"오는 11일 개최되는 ‘2021 MAMA’에서는 Stray Kids(스트레이 키즈) 현진, ATEEZ(에이티즈) 우영, ITZY(있지) 예지, 투모로우바이투게더 연준, aespa(에스파) 카리나, ENHYPEN(엔하이픈) 희승까지 K-POP 4세대 아이돌을 대표하는 각 그룹의 멤버 중 한 명씩 총 6명이 모여 오프닝 무대를 펼친다. 글로벌 무대에서도 사랑받고 있는 이들의 만남에 기대감이 높아지고 있다.",
"‘2021 MAMA’라는 글로벌 축제의 서막을 여는 중요한 무대인 만큼 스트레이 키즈 현진, 에이티즈 우영, 있지 예지, 투모로우바이투게더 연준, 에스파 카리나, 엔하이픈 희승은 완벽한 무대를 보여주기 위해 틈틈이 시간을 내 연습에 매진하고 있다는 후문이다. 각자의 개성을 살리면서도 서로의 매력이 조화를 이루어 완성될 오프닝 무대에 전 세계 팬들의 이목이 집중되고 있다.",
"K-POP 4세대 아이돌들의 에너지 넘치는 오프닝 무대로 또 하나의 레",
"전드 무대 탄생을 예고한 가운데, ‘2021 MAMA’는 오는 11일 오후 6시(KST)부터 Mnet 을 비롯해 미국과 아시아 각 지역의 채널과 플랫폼에 동시 생중계 된다. 또, Mnet K-POP 유튜브 채널을 포함한 4개 채널을 통해 전 세계 200여 국가에서 온라인으로 시청할 수 있다.",
"지난 10월5일 발매된 에스파 첫 미니앨범 '새비지'는 7일(현지시간) 발표된 미국 빌보드의 메인 차트 '빌보드 200'과 '아티스트 100'에서 각각 85위, 99위를 기록하며 발매 후 두 달이 지난 시점에 재진입, '롱런 인기'를 입증했다.",
"NCT 정규 3집 타이틀 곡 '유니버스(Let's Play Ball)'는 10일 오후 6시 각종 음악 사이트를 통해 발매된다. 뮤직비디오도 유튜브 SMTOWN 채널에서 동시 공개한다.",
"도영, 정우, 마크, 샤오쥔, 제노, 해찬, 재민, 양양, 쇼타로가 참여한 신곡 '유니버스(Let's Play Ball)'는 중독성 강한 훅이 매력적인 힙합 기반 R&B 댄스 곡이다. 가사에는 “너는 나의 세상이자 나를 움직이는 존재이고, 너라는 우주를 향해 힘차게 달려가는 과정이 아름답고 운명적”이라는 메시지를 담았다.",
"더불어 함께 공개되는 '유니버스(Let's Play Ball)' 뮤직비디오는 파워풀한 곡 분위기를 극대화하는 에너지 넘치는 퍼포먼스와 강렬한 카리스마가 돋보이는 멤버들 모습이 담겨 있어 음악 팬들 시선을 사로잡을 전망이다.",
"동의과학대에서 열린 후원금 기탁식에는 동의과학대 김영도 총장, 염",
"종석 야구부 감독, 일광남양엔시티 윤성천 대표 등 양 기관 대표자들이 참석했다.",
"동의과학대는 지난해 대학 스포츠 저변 확대를 위해 부산지역 전문대학 최초로 야구부와 축구부를 창단했다. 야구부는 창단 첫해 만에 '2021 KUSF 대학 야구 U-리그 왕중왕전'에 진출하는 성과를 거뒀다.",
"그룹 NCT 127(엔시티 127)이 정규 3집 '스티커(Sticker)'로 미국 '빌보드 200' 차트에 10주 연속 진입했다.",
"앞서 NCT 127은 이번 앨범으로 발매 첫 주 빌보드의 메인 차트 '빌보드 200' 및 '아티스트 100' 3위에 올랐으며, '톱 앨범 세일즈', '톱 커런트 앨범 세일즈', '월드 앨범', '인디펜던트 앨범' 등 빌보드 4개 차트에서 정상을 차지한 바 있다.",
"또한 NCT 127은 정규 3집으로 총 358만여 장의 음반 판매량을 기록해 트리플 밀리언셀러에 등극했다.",
"그의 준엄한 일갈을 듣고 싶어 사람들이 모여든다. 국내 1세대 힙합 래퍼", 
"원썬(본명 김선일·43)이 최근 ‘쇼미더머니10 리뷰’ 시리즈를 시작한 그의 유튜브 채널은 힙합 팬들의 성지로 거듭나고 있다.",
"부스터샷, 오미크론 변이 75% 예방…오미크론 우세종 가능성 多",
"'제한적 디폴트' 中 헝다 회장, 또 주식 매도…14억주 이상 팔아",
"서울 아파트 전세 26개월만에 ‘공급 〉수요’ 역전",
"[평화민주당 연구] 평민당이 지적한 '민자당 정권의 죄악상'",
"식용류로 인한 하수구 막힘 문제를 해결한 스타트업",
"아마도 대부분의 가정은 쓰고 남은 식용유를 하수구에 그냥 버리고 있을 것이다. 환경을 생",
"각해서 식용류를 키친타올로 닦아서 쓰레기통에 버리는 사람들은 그나마 잘 하고 있는 것이다.",
              "갖다 붙이기만 해도 뜨니…NFT 리스크는 따져봤나",
              "[일요신문] 디지털 자산 NFT(대체불가토큰) 열풍이 급속히 확산하면서 기대와 우려가 교",
              "차하고 있다. 게임과 엔터테인먼트, 미술 등 여러 산업의 성장 동력이 된다는 점은 긍정적",
              "이지만, 구매 시 법적 저작권을 보장하지 않고 범죄 수단으로 활용될 가능성이 높다는 점에",
              "서 우려의 시선도 감지된다. ",
              "재미로 봤다가 낭패볼 수도? 중국 2030 점술에 열광하는 까닭",
              "타오바이바이는 올해 7월부터 별자리 블로거 활동을 시작했다. 회원들이 태어난 날짜를 알",
              "‘실생활 고민 해결용’ 타로·별자리 등 폭발적 인기…사기 및 정보 유출 등",
              "리타이항은 “타로를 보는 행위는 생활 속에서 큰 위로를 준다. 스트레스가 심할 때 한 번씩",
              "명 중 1명은 점술 등에 대해 ‘믿는다’고 답했다. 또 운세를 직접 본 중국인은 40%에 달했",
              "동국대, 「108배 기부 릴레이」 캠페인 열어",
              "모금액은 전액 건학이념 구현 위한 재학생 장학금으로",
              "동국대(총장 윤성이)가 학교법인 건학위원회(위원장 돈관스님) 출범과 더불어 제2건학을 위한 새로운 100년의 기틀을 마련하고자 「108배 기부 릴레이」 캠페인을 연다.",
              "이데일리","세계일보","서울경제","동아일보","데일리안","헤럴드경제","뉴시스","미디어오늘","서울신문","연합뉴스TV","YTN","뉴스1","MBC","JTBC","SBS", "뉴스타파", "YTN", "Korea JoongAng Daily", "법률신문", "디자인정글", "일요신문", "스포츠서울", "OSEN", "한국일보", "CEO스코어데일리", "미디어오늘", "신아일보", "NEWSEN", "비즈한국", "초이스경제", "오마이뉴스", "CNN", ""]

In [4]:
# 광고 데이터와 비광고 데이터 병합
data = data_ad+data_nonad
print("광고 데이터 수 : ",len(data_ad))
print("비광고 데이터 수 : ",len(data_nonad))
print("총 데이터 수 : ",len(data))

광고 데이터 수 :  219
비광고 데이터 수 :  205
총 데이터 수 :  424


In [5]:
# 모델링 데이터 형태로 변환, 확인
x = np.array([1, 0])
column_name = np.repeat(x, [len(data_ad), len(data_nonad)], axis=0)
data_final = pd.DataFrame({'ad':column_name,'data':data})
data_final

,ad,data
0,1,이충동 즐거운점핑
1,1,재밌고 신나게 점핑해요
2,1,정말 이쁘게 살빠져요
3,1,남사면 마이그린
4,1,식물농장
...,...,...
419,0,비즈한국
420,0,초이스경제
421,0,오마이뉴스
422,0,CNN


In [6]:
# 데이터, 레이블 분리
X_data = data_final['data']
y_data = data_final['ad']

In [7]:
# train, test dataset 분리
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=0, stratify=y_data)

In [8]:
# 데이터 전처리([CLS], [SEP] 추가), 토큰화, 패딩
sentences = ["[CLS] " + str(s) + " [SEP]" for s in X_train]
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in sentences]
MAX_LEN = 128 
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [9]:
# attention mask 추가
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [10]:
# train, valid dataset 분리
labels = y_train.values
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2000, 
                                                                                    test_size=0.1)
                                                
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2000, 
                                                       test_size=0.1)     
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [11]:
# batch size 지정 후 모델링용 전처리 진행
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [12]:
# 동일한 과정으로 test dataset 전처리하기
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in X_test]

labels = y_test.values

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
    
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

batch_size = 32
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [13]:
# 사용할 장비 최종 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [14]:
# pretrained model 호출 후 구조 확인
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [15]:
# 모델 관련 정보(최적화 기법, 에폭, 훈련 스텝, 스케줄러) 설정
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)
epochs = 5
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [16]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)
    
    
# 시간 표시 함수
def format_time(elapsed):

    elapsed_rounded = int(round((elapsed)))
    
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [17]:
#랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#그래디언트 초기화
model.zero_grad()

# 학습
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
    
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 5 ========
Training...

  Average training loss: 0.56
  Training epcoh took: 0:00:01

Running Validation...
  Accuracy: 0.83
  Validation took: 0:00:00

======== Epoch 2 / 5 ========
Training...

  Average training loss: 0.34
  Training epcoh took: 0:00:01

Running Validation...
  Accuracy: 0.93
  Validation took: 0:00:00

======== Epoch 3 / 5 ========
Training...

  Average training loss: 0.17
  Training epcoh took: 0:00:01

Running Validation...
  Accuracy: 0.93
  Validation took: 0:00:00

======== Epoch 4 / 5 ========
Training...

  Average training loss: 0.10
  Training epcoh took: 0:00:01

Running Validation...
  Accuracy: 0.97
  Validation took: 0:00:00

======== Epoch 5 / 5 ========
Training...

  Average training loss: 0.07
  Training epcoh took: 0:00:01

Running Validation...
  Accuracy: 1.00
  Validation took: 0:00:00

Training complete!


In [18]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))



Accuracy: 0.95
Test took: 0:00:00


In [19]:
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [20]:
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [21]:
# 비광고 데이터로 실험
logits = test_sentences(["포스코그룹은 지주사전환을 기반으로 2030년까지 기업가치를 현재의 3배 이상으로 증대시킨다는 계획이다. △철강 △이차전지 소재 △리튬·니켈 △수소 △에너지 △건축·인프라 △식량(Agri-Bio) 등이 주축이다."])
print(logits)

if np.argmax(logits) == 1 :
    print("광고")
elif np.argmax(logits) == 0 :
    print("비광고")

[[ 1.5062335 -1.436984 ]]
비광고


In [22]:
# 광고(혹은 무관) 데이터로 실험
logits = test_sentences(['기사입력 2021.12.11. 오후 4:19'])
print(logits)

if np.argmax(logits) == 1 :
    print("광고")
elif np.argmax(logits) == 0 :
    print("비광고")

[[-0.8895308  1.2222911]]
광고


In [23]:
# model 저장
import torch
torch.save(model, "./classifi.ver2")